In [7]:
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
with open('imdb_movies_2000to2022.prolific.json', 'r') as f:
    imdb_data = [json.loads(line) for line in f]

In [9]:
imdb_rows = []
for entry in imdb_data:
    if "rating" in entry and "avg" in entry["rating"] and entry["rating"]["avg"] != "\\N":
        rating = float(entry["rating"]["avg"])
        year = int(entry["year"]) if entry["year"].isdigit() else None
        for actor in entry.get("actors", []):
            imdb_rows.append({
                "actor_name": actor[1],
                "rating": rating,
                "year": year
            })

df_imdb = pd.DataFrame(imdb_rows)